In [15]:
from typing import List
from typing import Tuple

def parse_board_input(list: List[str], num_rows=5) -> List[List[List[int]]]:
    boards: List[List[int]] = []
    cur_board = 0
    for row in list:
        if len(boards) == cur_board:
            boards.append([])
        if row == "":
            continue
        boards[cur_board].append([int(x) for x in row.split(" ") if x != ""])
        if len(boards[cur_board]) == num_rows:
            cur_board += 1
    return boards

In [16]:
with open("example_boards.txt", "r") as input_file:
    input_raw = input_file.read()

lines = input_raw.splitlines()
example_boards = parse_board_input(lines)

with open("example_draws.txt", "r") as input_file:
    example_draws = [int(x) for x in input_file.read().split(",")]
print(example_draws)

[7, 4, 9, 5, 11, 17, 23, 2, 0, 14, 21, 24, 10, 16, 13, 6, 15, 25, 12, 22, 18, 20, 8, 19, 3, 26, 1]


In [17]:
class BingoNumber():
    value: int
    marked: bool = False
    def __init__(self, value: int):
        self.value = value
    def __str__(self) -> str:
        string = str(self.value)
        if self.marked:
            string = "~" + str(self.value) +"~"
        return string

class BingoBoard():
    rows: List[List[BingoNumber]]
    won: bool
    board_num: int
    def __init__(self, board: List[List[int]], board_num: int):
        self.rows = []
        self.won = False
        self.board_num = board_num
        for row in board:
            brow = []
            for num in row:
                brow.append(BingoNumber(num))
            self.rows.append(brow)
        
    def draw(self, num: int) -> None:
        for row in self.rows:
            for x in row:
                if x.value == num and not self.won:
                    x.marked = True

    def check_rows(self) -> bool:
        if self.won:
            return False
        for row in self.rows:
            found_false = False
            for x in row:
                if not x.marked:
                    found_false = True
            if not found_false:
                return True
        return False

    def check_columns(self) -> bool:
        if self.won:
            return False
        for c in range(5):
            found_false = False
            for row in self.rows:
                if not row[c].marked:
                    found_false = True
            if not found_false:
                return True
        return False

    def check(self) -> bool:
        return self.check_rows() or self.check_columns()
    
    def sum_unmarked(self) -> int:
        sum = 0
        for row in self.rows:
            for x in row:
                if not x.marked:
                    sum += x.value
        return sum

    def score(self, winning_draw: int) -> int:
        sum = self.sum_unmarked()
        return sum * winning_draw
    
    def reset(self) -> None:
        self.won = False
        for row in self.rows:
            for x in row:
                x.marked = False

    def __str__(self) -> str:
        string = ""
        for row in self.rows:
            string += "["
            i = 0
            for num in row:
                string += str(num)
                if i != len(row)-1:
                    string += '\t'
                i += 1
            string +=  "],\n"
        return string

def create_boards(d: List[List[List[int]]]) -> List[BingoBoard]:
    board_num = 1
    boards = []
    for b in d:
        boards.append(BingoBoard(b, board_num))
        board_num += 1
    return boards

In [18]:
def draw_pos(boards: List[BingoBoard], draws: List[int], pos: int) -> Tuple[BingoBoard, int]:
    won: Tuple[BingoBoard, int] = None
    for board in boards:
        board.draw(draws[pos])
        if not board.won and board.check():
            board.won = True
            won = (board, draws[pos])
    return won

def draw_n_pos(boards: List[BingoBoard], draws: List[int], n: int) -> List[Tuple[BingoBoard, int]]:
    winners = []
    reset_boards(boards)
    for x in range(n):
        winner = draw_pos(boards, draws, x)
        if winner:
            winners.append((winner[0], winner[0].score(winner[1])))
    return winners

def print_boards(boards: List[BingoBoard]) -> None:
    for b in boards:
        print(b)

def find_first_winner(boards: List[BingoBoard], draws: List[int]) -> Tuple[BingoBoard, int]:
    return draw_n_pos(boards, draws, len(draws))[0]

def find_last_winner(boards: List[BingoBoard], draws: List[int]) -> Tuple[BingoBoard, int]:
    return draw_n_pos(boards, draws, len(draws))[-1]
    
def reset_boards(boards: List[BingoBoard]) -> None:
    for board in boards:
        board.reset()

In [19]:
boards = create_boards(example_boards)
winner = find_first_winner(boards, example_draws)
print(winner[1])

last_winner = find_last_winner(boards, example_draws)
print(last_winner[1])


4512
1924


In [20]:
with open("input_boards.txt", "r") as input_file:
    input_raw = input_file.read()

lines = input_raw.splitlines()
input_boards = parse_board_input(lines)

with open("input_draws.txt", "r") as input_file:
    input_draws = [int(x) for x in input_file.read().split(",")]

In [23]:
boards = create_boards(input_boards)
winner = find_first_winner(boards, input_draws)
print(winner[1])

last_winner = find_last_winner(boards, input_draws)
print(last_winner[1])

2496
25925
